In [1]:
#Baixando a base de dados para a variável x

import pandas as pd
x = pd.read_csv(
    filepath_or_buffer='https://noverde-data-engineering-test.s3.amazonaws.com/dataeng_test.csv',
    sep=',')

In [2]:
#Excluindo a coluna que apresenta apenas valores nulos e passando os dados para a variável df

df = x.drop('Unnamed: 4', axis=1)

In [3]:
#Tratando a coluna 'ValorPago' - Removendo primeiros os '.'

df['ValorPago'] = df['ValorPago'].apply(lambda x: x.replace('.', ''))

In [4]:
#Tratando a coluna 'ValorPago' - Removendo as vírgulas e substituindo para pontos

df['ValorPago'] = df['ValorPago'].apply(lambda x: x.replace(',', '.'))

In [5]:
#Transformando a coluna 'ValorPago' em float

df['ValorPago']=df['ValorPago'].astype('float64')

In [6]:
#Transformando a coluna 'Comarca' - Removendo ' / SP'

df['Comarca'] = df['Comarca'].apply(lambda x:x.replace(' / SP',''))

In [7]:
#Baixando o CSV

df.to_csv('C:\Files\protestos.csv',index=False)

In [8]:
#Importando o adaptador do PostgreSQL para python
import psycopg2

#Criando a conexão com o PostgreSQL
conex = psycopg2.connect(host='localhost', database='Noverde',
user='postgres', password='dnr@2016')

#Abrindo o cursor para criação da tabela "payments"
cur = conex.cursor()

In [9]:
#Criando a tabela "payments"

cur.execute("CREATE TABLE payments (id serial PRIMARY KEY, payment_date date, amount decimal(10,2),protocol varchar(7),city varchar(30), state varchar(2) DEFAULT 'sp');")

In [10]:
# Inserindo dados na tabela "payments"

cur.execute("COPY payments (protocol,city,payment_date,amount) FROM 'C:\Files\protestos.csv' DELIMITER ',' CSV HEADER;")

In [11]:
# Query 1 
#Qual foi o valor recebido em todo o período?

cur.execute("SELECT cast(sum(amount) as money) as ValorRecebido FROM payments;")
cur.fetchone()

('R$ 88.792,04',)

In [12]:
#Query 2
#Qual o maior e o menor valor recebido em 2019?

cur.execute("SELECT cast(max(amount) as money) as MaiorValor, cast(min(amount) as money) as MenorValor FROM payments WHERE payment_date BETWEEN '20190101' AND '20191231';")
cur.fetchall()

[('R$ 3.676,89', 'R$ 123,13')]

In [13]:
#Query 3
#Qual o mês que teve o maior recebimento?

cur.execute("SELECT EXTRACT('YEAR' FROM payment_date) AS Ano, EXTRACT('MONTH' FROM payment_date) AS Mes, CAST(sum(amount)AS MONEY) AS Valortotal FROM payments GROUP BY EXTRACT('MONTH' FROM payment_date), EXTRACT('YEAR' FROM payment_date) ORDER BY Valortotal desc limit 1;")
cur.fetchall()

[(2019.0, 5.0, 'R$ 19.969,83')]

In [14]:
#Query 4
#Qual a comarca que teve mais recebimento?

cur.execute("SELECT city AS Comarca, CAST(sum(amount)AS MONEY) AS Valor FROM payments GROUP BY city ORDER BY Valor desc limit 1;")
cur.fetchall()

[('SAO PAULO', 'R$ 17.366,13')]

In [15]:
#Query 5
# Qual a porcentagem que cada mês de 2019 teve no montante recebido de 2019? 

cur.execute("SELECT EXTRACT('MONTH' from payment_date) AS month, EXTRACT('YEAR' from payment_date) AS year, cast(sum(amount)as money) as amount, cast(sum(amount)/(SELECT sum(amount) from payments WHERE EXTRACT('YEAR' from payment_date) = 2019)*100 as decimal(10,2)) as Ratio FROM payments WHERE EXTRACT('YEAR' from payment_date) = 2019 GROUP BY EXTRACT('MONTH' FROM payment_date), EXTRACT('YEAR' from payment_date) ORDER BY month asc;")
cur.fetchall()

[(1.0, 2019.0, 'R$ 5.671,26', Decimal('7.62')),
 (2.0, 2019.0, 'R$ 5.868,78', Decimal('7.89')),
 (3.0, 2019.0, 'R$ 17.942,57', Decimal('24.12')),
 (4.0, 2019.0, 'R$ 8.179,58', Decimal('11.00')),
 (5.0, 2019.0, 'R$ 19.969,83', Decimal('26.85')),
 (6.0, 2019.0, 'R$ 16.752,40', Decimal('22.52'))]

In [17]:
#Fazendo as alterações no banco

conex.commit ()

In [18]:
#Fechando cursor e conexão

cur.close ()
conex.close ()

In [ ]:
#Fim